In [4]:
# Import thư viện
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from sklearn.metrics import classification_report

In [5]:
# Khởi tạo SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()

In [6]:
# Đọc dữ liệu
train_pd = spark.read.parquet("train_set").toPandas()
test_pd = spark.read.parquet("test_set").toPandas()
print(f"Train size: {len(train_pd)}, Test size: {len(test_pd)}")

Train size: 3212, Test size: 804


In [7]:
# Load PhoBERT
device = torch.device("cuda")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
model = AutoModel.from_pretrained("vinai/phobert-base").to(device)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(64001, 768, padding_idx=1)
    (position_embeddings): Embedding(258, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [8]:
# Hàm xử lý batch embedding
def batch_embed(texts, batch_size=64):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        tokens = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=256)
        input_ids = tokens["input_ids"].to(device)
        attention_mask = tokens["attention_mask"].to(device)
        with torch.no_grad():
            output = model(input_ids, attention_mask=attention_mask)
            emb = output.last_hidden_state.max(dim=1)[0].cpu().numpy()
            all_embeddings.extend(emb)
    return all_embeddings

  # Vector hóa
train_embeddings = batch_embed(train_pd["review"].tolist())
test_embeddings = batch_embed(test_pd["review"].tolist())

In [9]:
# Chuyển numpy array thành Spark Vectors
train_pd["features"] = [Vectors.dense(emb) for emb in train_embeddings]
test_pd["features"] = [Vectors.dense(emb) for emb in test_embeddings]

In [10]:
# Chuyển từ Pandas sang Spark DataFrame
train_df = spark.createDataFrame(train_pd)
test_df = spark.createDataFrame(test_pd)

In [11]:
# Encode sentiment
indexer = StringIndexer(inputCol="sentiment", outputCol="label_idx")
indexer_model = indexer.fit(train_df)
train_df = indexer_model.transform(train_df)
test_df = indexer_model.transform(test_df)

In [12]:
# Định nghĩa mô hình Logistic Regression
lr = LogisticRegression(featuresCol="features", labelCol="label_idx")

In [21]:
# Xây dựng lưới tham số
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.maxIter, [10, 20]) \
    .build()

# Đánh giá mô hình
evaluator = MulticlassClassificationEvaluator(labelCol="label_idx", predictionCol="prediction", metricName="accuracy")

# Định nghĩa CrossValidator
cv = CrossValidator(estimator=lr,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3)

# Huấn luyện mô hình tốt nhất
cv_model = cv.fit(train_df)

In [22]:
# Chọn mô hình tốt nhất và in hyperparameter
best_model = cv_model.bestModel
best_params = best_model.extractParamMap()

print("\nHyperparameter tốt nhất tìm được:")
for param, value in best_params.items():
    if param.name in ["regParam", "maxIter", "elasticNetParam"]:
        print(f"  {param.name}: {value}")


Hyperparameter tốt nhất tìm được:
  elasticNetParam: 0.0
  maxIter: 20
  regParam: 0.1


In [23]:
# Dự đoán và đánh giá
predictions = best_model.transform(test_df)
accuracy = evaluator.evaluate(predictions)
print(f"\nĐộ chính xác trên tập test: {accuracy:.4f}")


Độ chính xác trên tập test: 0.9241


In [24]:
# Phân tích chi tiết
preds_pd = predictions.select("label_idx", "prediction").toPandas()
print("\nBáo cáo phân loại theo từng lớp:")
print(classification_report(preds_pd["label_idx"], preds_pd["prediction"], digits=4))

# Hiển thị mapping giữa label index và sentiment gốc
label_mapping = dict(enumerate(indexer_model.labels))
print("\nMapping label_idx -> sentiment:")
for idx, label in label_mapping.items():
    print(f"  {idx} = {label}")


Báo cáo phân loại theo từng lớp:
              precision    recall  f1-score   support

         0.0     0.9514    0.9759    0.9635       622
         1.0     0.8242    0.8947    0.8580       152
         2.0     0.0000    0.0000    0.0000        30

    accuracy                         0.9241       804
   macro avg     0.5919    0.6235    0.6072       804
weighted avg     0.8919    0.9241    0.9076       804


Mapping label_idx -> sentiment:
  0 = positive
  1 = negative
  2 = neutral


In [25]:
# Hiển thị một số dự đoán
result = predictions.select("review", "sentiment", "prediction").limit(5)
result.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+----------+
|review                                                                                                                                                                    |sentiment|prediction|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+----------+
|3 miếng gà 105k như này khác gì 2 miếng rưỡi đâu chứ                                                                                                                      |negative |1.0       |
|Gà giòn_tan , nóng_hổi , ngon . Các bạn nhân_viên thì dễ_thương , nhanh_nhẹn .                                                                                            |positive |0.0       |
|Gà giòn_rụm , thấm vị , bên t

In [ ]:
# Lưu mô hình
#lr_model.save("lr_sentiment_model")


Độ chính xác trên tập test: 0.9279
